<h1><div align="center">Managing Accelerated Application Memory with CUDA C/C++ Unified Memory</div></h1>

![CUDA](./images/CUDA_Logo.jpg)

The [*CUDA Best Practices Guide*](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations), a highly recommended followup to this and other CUDA fundamentals labs, recommends a design cycle called **APOD**: **A**ssess, **P**arallelize, **O**ptimize, **D**eploy. In short, APOD prescribes an iterative design process, where developers can apply incremental improvements to their accelerated application's performance, and ship their code. As developers become more competent CUDA programmers, more advanced optimization techniques can be applied to their accelerated code bases.

This lab will support such a style of iterative development. You will be using the Nsight Systems command line tool **nsys** to qualitatively measure your application's performance, and to identify opportunities for optimization, after which you will apply incremental improvements before learning new techniques and repeating the cycle. As a point of focus, many of the techniques you will be learning and applying in this lab will deal with the specifics of how CUDA's **Unified Memory** works. Understanding Unified Memory behavior is a fundamental skill for CUDA developers, and serves as a prerequisite to many more advanced memory management techniques.

---
## Prerequisites

To get the most out of this lab you should already be able to:

- Write, compile, and run C/C++ programs that both call CPU functions and launch GPU kernels.
- Control parallel thread hierarchy using execution configuration.
- Refactor serial loops to execute their iterations in parallel on a GPU.
- Allocate and free Unified Memory.

---
## Objectives

By the time you complete this lab, you will be able to:

- Use the Nsight Systems command line tool (**nsys**) to profile accelerated application performance.
- Leverage an understanding of **Streaming Multiprocessors** to optimize execution configurations.
- Understand the behavior of **Unified Memory** with regard to page faulting and data migrations.
- Use **asynchronous memory prefetching** to reduce page faults and data migrations for increased performance.
- Employ an iterative development cycle to rapidly accelerate and deploy applications.

---
## Iterative Optimizations with the NVIDIA Command Line Profiler

The only way to be assured that attempts at optimizing accelerated code bases are actually successful is to profile the application for quantitative information about the application's performance. `nsys` is the Nsight Systems command line tool. It ships with the CUDA toolkit, and is a powerful tool for profiling accelerated applications.

`nsys` is easy to use. Its most basic usage is to simply pass it the path to an executable compiled with `nvcc`. `nsys` will proceed to execute the application, after which it will print a summary output of the application's GPU activities, CUDA API calls, as well as information about **Unified Memory** activity, a topic which will be covered extensively later in this lab.

When accelerating applications, or optimizing already-accelerated applications, take a scientific and iterative approach. Profile your application after making changes, take note, and record the implications of any refactoring on performance. Make these observations early and often: frequently, enough performance boost can be gained with little effort such that you can ship your accelerated application. Additionally, frequent profiling will teach you how specific changes to your CUDA code bases impact its actual performance: knowledge that is hard to acquire when only profiling after many kinds of changes in your code bases.

### Exercise: Profile an Application with nsys

[01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) (<------ you can click on this and any of the source file links in this lab to open them for editing) is a naively accelerated vector addition program. Use the two code execution cells below (`CTRL` + `ENTER`). The first code execution cell will compile (and run) the vector addition program. The second code execution cell will profile the executable that was just compiled using `nsys profile`.

`nsys profile` will generate a `qdrep` report file which can be used in a variety of manners. We use the `--stats=true` flag here to indicate we would like summary statistics printed. There is quite a lot of information printed:

- Profile configuration details
- Report file(s) generation details
- **CUDA API Statistics**
- **CUDA Kernel Statistics**
- **CUDA Memory Operation Statistics (time and size)**
- OS Runtime API Statistics

In this lab you will primarily be using the 3 sections in **bold** above. In the next lab, you will be using the generated report files to give to the Nsight Systems GUI for visual profiling.

After profiling the application, answer the following questions using information displayed in the `CUDA Kernel Statistics` section of the profiling output:

- What was the name of the only CUDA kernel called in this application? addVectorsInto
- How many times did this kernel run? Once
- How long did it take this kernel to run? Record this time somewhere: you will be optimizing this application and will want to know how much faster you can make it.

Total time: 2284689086ns (2.28468909s)

In [1]:
!nvcc -o single-thread-vector-add 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


In [2]:
!nsys profile --stats=true ./single-thread-vector-add

Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-b93d-e79c-67c2-8f5f.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-b93d-e79c-67c2-8f5f.qdrep"
Exporting 4623 events: [==================================================100%][===========26%                                        ]

Exported successfully to
/tmp/nsys-report-b93d-e79c-67c2-8f5f.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average      Minimum     Maximum            Name         
 -------  ---------------  ---------  ------------  ----------  ----------  ---------------------
    88.0       2284697703          1  2284697703.0  2284697703  2284697703  cudaDeviceSynchronize
    11.2        290305345          3    96768448.3       38439   290202301  cudaMallocManaged    
     0.8         21054397          3     7018132.3     631

Worth mentioning is that by default, `nsys profile` will not overwrite an existing report file. This is done to prevent accidental loss of work when profiling. If for any reason, you would rather overwrite an existing report file, say during rapid iterations, you can provide the `-f` flag to `nsys profile` to allow overwriting an existing report file.

### Exercise: Optimize and Profile

Take a minute or two to make a simple optimization to [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) by updating its execution configuration so that it runs on many threads in a single thread block. Recompile and then profile with `nsys profile --stats=true` using the code execution cells below. Use the profiling output to check the runtime of the kernel. What was the speed up from this optimization? Be sure to record your results somewhere.

With threadsPerBlock = 128 and numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock):

    total time: 153630818ns = 0.15363082s (10x speedup)
    
with threadsPerBlock = 32:

    total time: 150430974ns = 0.15043097 (10x speedup)

In [5]:
!nvcc -o multi-thread-vector-add 01-vector-add/01-vector-add.cu -run

Success! All values calculated correctly.


In [6]:
!nsys profile --stats=true ./multi-thread-vector-add

Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-da44-3944-974d-dba7.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-da44-3944-974d-dba7.qdrep"
Exporting 4729 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-da44-3944-974d-dba7.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    58.7        243830780          3   81276926.7      18343  243754977  cudaMallocManaged    
    36.2        150441765          1  150441765.0  150441765  150441765  cudaDeviceSynchronize
     5.1         20989387          3    6996462.3    6302934    8136145  cudaFree             
     0.0            53401      

### Exercise: Optimize Iteratively

In this exercise you will go through several cycles of editing the execution configuration of [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu), profiling it, and recording the results to see the impact. Use the following guidelines while working:

- Start by listing 3 to 5 different ways you will update the execution configuration, being sure to cover a range of different grid and block size combinations.
- Edit the [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program in one of the ways you listed.
- Compile and profile your updated code with the two code execution cells below.
- Record the runtime of the kernel execution, as given in the profiling output.
- Repeat the edit/profile/record cycle for each possible optimization you listed above

Which of the execution configurations you attempted proved to be the fastest?

- Grid size: 4194304, Block size: 8, total time: 152898071ns
- Grid size: 1048576, Block size: 32, total time: 151239831ns
- Grid size: 262144, Block size: 128, total time: 141898366ns
- Grid size: 65536, Block size: 512, total time: 147954972ns
- Grid size: 32768, Block size: 1024, total time: as low as 125480675ns, as high as 145576306ns

The largest block size runs the fastest.

In [19]:
!nvcc -o iteratively-optimized-vector-add 01-vector-add/01-vector-add.cu -run

Grid size: 32768, Block size: 1024
Success! All values calculated correctly.


In [22]:
!nsys profile --stats=true ./iteratively-optimized-vector-add

Grid size: 32768, Block size: 1024
Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-6de6-ef83-5f52-f16f.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-6de6-ef83-5f52-f16f.qdrep"
Exporting 10163 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-6de6-ef83-5f52-f16f.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    62.9        283487506          3   94495835.3      17821  283414634  cudaMallocManaged    
    32.3        145587347          1  145587347.0  145587347  145587347  cudaDeviceSynchronize
     4.7         21212950          3    7070983.3    6361623    8223294  cudaFree          

---
## Streaming Multiprocessors and Querying the Device

This section explores how understanding a specific feature of the GPU hardware can promote optimization. After introducing **Streaming Multiprocessors**, you will attempt to further optimize the accelerated vector addition program you have been working on.

The following slides present upcoming material visually, at a high level. Click through the slides before moving on to more detailed coverage of their topics in following sections.

In [23]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/embedded/task2/NVPROF_UM_1.pptx" width="800px" height="500px" frameborder="0"></iframe></div>

### Streaming Multiprocessors and Warps

The GPUs that CUDA applications run on have processing units called **streaming multiprocessors**, or **SMs**. During kernel execution, blocks of threads are given to SMs to execute. In order to support the GPU's ability to perform as many parallel operations as possible, performance gains can often be had by *choosing a grid size that has a number of blocks that is a multiple of the number of SMs on a given GPU.*

Additionally, SMs create, manage, schedule, and execute groupings of 32 threads from within a block called **warps**. A more [in depth coverage of SMs and warps](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation) is beyond the scope of this course, however, it is important to know that performance gains can also be had by *choosing a block size that has a number of threads that is a multiple of 32.*

### Programmatically Querying GPU Device Properties

In order to support portability, since the number of SMs on a GPU can differ depending on the specific GPU being used, the number of SMs should not be hard-coded into a code bases. Rather, this information should be acquired programatically.

The following shows how, in CUDA C/C++, to obtain a C struct which contains many properties about the currently active GPU device, including its number of SMs:

```cpp
int deviceId;
cudaGetDevice(&deviceId);                  // `deviceId` now points to the id of the currently active GPU.

cudaDeviceProp props;
cudaGetDeviceProperties(&props, deviceId); // `props` now has many useful properties about
                                           // the active GPU device.
```

### Exercise: Query the Device

Currently, [`01-get-device-properties.cu`](../edit/04-device-properties/01-get-device-properties.cu) contains many unassigned variables, and will print gibberish information intended to describe details about the currently active GPU.

Build out [`01-get-device-properties.cu`](../edit/04-device-properties/01-get-device-properties.cu) to print the actual values for the desired device properties indicated in the source code. In order to support your work, and as an introduction to them, use the [CUDA Runtime Docs](http://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html) to help identify the relevant properties in the device props struct. Refer to [the solution](../edit/04-device-properties/solutions/01-get-device-properties-solution.cu) if you get stuck.

In [26]:
!nvcc -o get-device-properties 04-device-properties/01-get-device-properties.cu -run

Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32


### Exercise: Optimize Vector Add with Grids Sized to Number of SMs

Utilize your ability to query the device for its number of SMs to refactor the `addVectorsInto` kernel you have been working on inside [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) so that it launches with a grid containing a number of blocks that is a multiple of the number of SMs on the device.

Depending on other specific details in the code you have written, this refactor may or may not improve, or significantly change, the performance of your kernel. Therefore, as always, be sure to use `nsys profile` so that you can quantitatively evaluate performance changes. Record the results with the rest of your findings thus far, based on the profiling output.

- Grid size: 40, Block size: 1024, total time: 135458141ns

In [27]:
!nvcc -o sm-optimized-vector-add 01-vector-add/01-vector-add.cu -run

Grid size: 40, Block size: 1024
Success! All values calculated correctly.


In [28]:
!nsys profile --stats=true ./sm-optimized-vector-add

Grid size: 40, Block size: 1024
Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-f4d1-1461-fef4-4779.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-f4d1-1461-fef4-4779.qdrep"
Exporting 14140 events: [=================================================100%]7%                                                  ]

Exported successfully to
/tmp/nsys-report-f4d1-1461-fef4-4779.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    62.8        264522004          3   88174001.3      17757  264437766  cudaMallocManaged    
    32.2        135467044          1  135467044.0  135467044  135467044  cudaDeviceSynchronize
     5.0         21119375          3    7

---
## Unified Memory Details

You have been allocating memory intended for use either by host or device code with `cudaMallocManaged` and up until now have enjoyed the benefits of this method - automatic memory migration, ease of programming - without diving into the details of how the **Unified Memory** (**UM**) allocated by `cudaMallocManaged` actual works.

`nsys profile` provides details about UM management in accelerated applications, and using this information, in conjunction with a more-detailed understanding of how UM works, provides additional opportunities to optimize accelerated applications.

The following slides present upcoming material visually, at a high level. Click through the slides before moving on to more detailed coverage of their topics in following sections.

In [29]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/embedded/task2/NVPROF_UM_2.pptx" width="800px" height="500px" frameborder="0"></iframe></div>

### Unified Memory Migration

When UM is allocated, the memory is not resident yet on either the host or the device. When either the host or device attempts to access the memory, a [page fault](https://en.wikipedia.org/wiki/Page_fault) will occur, at which point the host or device will migrate the needed data in batches. Similarly, at any point when the CPU, or any GPU in the accelerated system, attempts to access memory not yet resident on it, page faults will occur and trigger its migration.

The ability to page fault and migrate memory on demand is tremendously helpful for ease of development in your accelerated applications. Additionally, when working with data that exhibits sparse access patterns, for example when it is impossible to know which data will be required to be worked on until the application actually runs, and for scenarios when data might be accessed by multiple GPU devices in an accelerated system with multiple GPUs, on-demand memory migration is remarkably beneficial.

There are times - for example when data needs are known prior to runtime, and large contiguous blocks of memory are required - when the overhead of page faulting and migrating data on demand incurs an overhead cost that would be better avoided.

Much of the remainder of this lab will be dedicated to understanding on-demand migration, and how to identify it in the profiler's output. With this knowledge you will be able to reduce the overhead of it in scenarios when it would be beneficial.

### Exercise: Explore UM Migration and Page Faulting

`nsys profile` provides output describing UM behavior for the profiled application. In this exercise, you will make several modifications to a simple application, and make use of `nsys profile` after each change, to explore how UM data migration behaves.

[`01-page-faults.cu`](../edit/06-unified-memory-page-faults/01-page-faults.cu) contains a `hostFunction` and a `gpuKernel`, both which could be used to initialize the elements of a `2<<24` element vector with the number `1`. Currently neither the host function nor GPU kernel are being used.

For each of the 4 questions below, given what you have just learned about UM behavior, first hypothesize about what kind of page faulting should happen, then, edit [`01-page-faults.cu`](../edit/06-unified-memory-page-faults/01-page-faults.cu) to create a scenario, by using one or both of the 2 provided functions in the code bases, that will allow you to test your hypothesis.

In order to test your hypotheses, compile and profile your code using the code execution cells below. Be sure to record your hypotheses, as well as the results, obtained from `nsys profile --stats=true` output. In the output of `nsys profile --stats=true` you should be looking for the following:

- Is there a _CUDA Memory Operation Statistics_ section in the output?
- If so, does it indicate host to device (HtoD) or device to host (DtoH) migrations?
- When there are migrations, what does the output say about how many _Operations_ there were? If you see many small memory migration operations, this is a sign that on-demand page faulting is occurring, with small memory migrations occurring each time there is a page fault in the requested location.

Here are the scenarios for you to explore, along with solutions for them if you get stuck:

- Is there evidence of memory migration and/or page faulting when unified memory is accessed only by the CPU? ([solution](../edit/06-unified-memory-page-faults/solutions/01-page-faults-solution-cpu-only.cu))
    - **Hypothesis**: no page faults/migration,
    - **Result**: no CUDA Memory Operation Statistics section at all
- Is there evidence of memory migration and/or page faulting when unified memory is accessed only by the GPU? ([solution](../edit/06-unified-memory-page-faults/solutions/02-page-faults-solution-gpu-only.cu))
    - **Hypothesis**: page fault and migration of vector in probably small chunks from host to device
    - **Result**: also no CUDA Memory Operation Statistics section at all, I guess it makes a copy on both initially?
- Is there evidence of memory migration and/or page faulting when unified memory is accessed first by the CPU then the GPU? ([solution](../edit/06-unified-memory-page-faults/solutions/03-page-faults-solution-cpu-then-gpu.cu))
    - **Hypothesis**: no migration/fault when running on host, then migration and fault when running on GPU, host to device
    - **Result**: 768 host to device memcpy operations
- Is there evidence of memory migration and/or page faulting when unified memory is accessed first by the GPU then the CPU? ([solution](../edit/06-unified-memory-page-faults/solutions/04-page-faults-solution-gpu-then-cpu.cu))
    - **Hypothesis**: no page faults/migration when first run on GPU, then 768 device to host memcpy operations
    - **Results**: the hypothesis

In [40]:
!nvcc -o page-faults 06-unified-memory-page-faults/01-page-faults.cu -run

In [41]:
!nsys profile --stats=true ./page-faults

Processing events...
Saving temporary "/tmp/nsys-report-7abf-cfec-37cb-1c40.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-7abf-cfec-37cb-1c40.qdrep"
Exporting 1977 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-7abf-cfec-37cb-1c40.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    76.8        839136722          1  839136722.0  839136722  839136722  cudaDeviceSynchronize
    22.4        244975265          1  244975265.0  244975265  244975265  cudaMallocManaged    
     0.8          8493666          1    8493666.0    8493666    8493666  cudaFree             
     0.0            65433          1      65433.0      65433      65433  

### Exercise: Revisit UM Behavior for Vector Add Program

Returning to the [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program you have been working on throughout this lab, review the code bases in its current state, and hypothesize about what kinds of memory migrations and/or page faults you expect to occur. Look at the profiling output for your last refactor (either by scrolling up to find the output or by executing the code execution cell just below), observing the _CUDA Memory Operation Statistics_ section of the profiler output. Can you explain the kinds of migrations and the number of their operations based on the contents of the code base?

Lots of host to device migrations, followed by a few device to host migrations.
The vectors are allocated using `cudaMallocManaged()`, then initialized with `initWith()`, which is performed on the host.
Since the kernel is called next, we need to send the three initialized vectors in their entirety to the device (thus the large number of memcpy HtoD)
Finally, we have to verify the elements with `checkElementsAre()`, which requires passing back single result vector.

If we look at the total size in KiB:

- HtoD: `393216 KiB`
- DtoH: `131072 KiB`

Sure enough, we're passing three vectors worth of KiB to the device for the kernel, and one vector worth of KiB back to the device for the verification.
If we look at the memory operation statistics by time, we can see that the HtoD operations take four times as long as the DtoH operations, despite only technically needing to move three times as many bytes.
I suspect this is because the GPU is requesting a page of vector `a`, not getting it, having a page moved, then requesting a page of vector `b`, not getting it, requesting it, etc.
Since we're adding the first element of each vector, then the second, we can really only grab a single value in each memcpy, and so have lots of page faults and need a large number of memcpy operations.
In contrast, when copying the result vector back, we know that we're grabbing from the same contiguous block of device memory (since the result vector is presumably contiguous), so we can grab the largest pages possible on a page fault.
Basically, cache coherence, but for unified memory.

In [42]:
!nsys profile --stats=true ./sm-optimized-vector-add

Grid size: 40, Block size: 1024
Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-5a8e-a956-5183-0f66.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-5a8e-a956-5183-0f66.qdrep"
Exporting 16097 events: [=================================================100%]

Exported successfully to
/tmp/nsys-report-5a8e-a956-5183-0f66.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls    Average     Minimum    Maximum           Name         
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    59.4        243445698          3   81148566.0      18717  243365848  cudaMallocManaged    
    35.4        145373893          1  145373893.0  145373893  145373893  cudaDeviceSynchronize
     5.2         21262271          3    7087423.7    6404463    8196903  cudaFree             

### Exercise: Initialize Vector in Kernel

When `nsys profile` gives the amount of time that a kernel takes to execute, the host-to-device page faults and data migrations that occur during this kernel's execution are included in the displayed execution time.

With this in mind, refactor the `initWith` host function in your [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program to instead be a CUDA kernel, initializing the allocated vector in parallel on the GPU. After successfully compiling and running the refactored application, but before profiling it, hypothesize about the following:

- How do you expect the refactor to affect UM memory migration behavior?
    - **Hypothesis**: with the first operation performed on each vector done on the device, we won't have any HtoD memcpy ops. There will still be the DtoH ops when verifying the result, as that is not yet a kernel.
    - **Results**: hypothesis confirmed, no HtoD operations.
- How do you expect the refactor to affect the reported run time of `addVectorsInto`?
    - **Hypothesis**: runtime will be reduced by about the total time of all the HtoD memcpy ops, or about 88268453ns, bringing the total runtime to 57095882ns, minus overhead caused by additional operations in the kernel required to get the stride and index (and runtime of `addVectorsInto` will be reduced by the time the memcpy would usually take)
    - **Results**: Total time reduced to about 57939902ns, which is about what I guess, minus overhead. `addVectorsInto`'s execution time shrank drastically, again by approximately the time required to move memory to the device.

Once again, record the results. Refer to [the solution](../edit/07-init-in-kernel/solutions/01-vector-add-init-in-kernel-solution.cu) if you get stuck.

In [46]:
!nvcc -o initialize-in-kernel 01-vector-add/01-vector-add.cu -run

Grid size: 40, Block size: 1024
Success! All values calculated correctly.


In [47]:
!nsys profile --stats=true ./initialize-in-kernel

Grid size: 40, Block size: 1024
Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-e732-809c-9869-13a6.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-e732-809c-9869-13a6.qdrep"
Exporting 1885 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-e732-809c-9869-13a6.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name         
 -------  ---------------  ---------  ----------  -------  ---------  ---------------------
    76.8        260194106          3  86731368.7    24586  260102732  cudaMallocManaged    
    17.1         57931762          2  28965881.0  1886605   56045157  cudaDeviceSynchronize
     6.1         20550288          3   6850096.0  6065628    8262513  cudaFree             
     0.0      

---
## Asynchronous Memory Prefetching

A powerful technique to reduce the overhead of page faulting and on-demand memory migrations, both in host-to-device and device-to-host memory transfers, is called **asynchronous memory prefetching**. Using this technique allows programmers to asynchronously migrate unified memory (UM) to any CPU or GPU device in the system, in the background, prior to its use by application code. By doing this, GPU kernels and CPU function performance can be increased on account of reduced page fault and on-demand data migration overhead.

Prefetching also tends to migrate data in larger chunks, and therefore fewer trips, than on-demand migration. This makes it an excellent fit when data access needs are known before runtime, and when data access patterns are not sparse.

CUDA Makes asynchronously prefetching managed memory to either a GPU device or the CPU easy with its `cudaMemPrefetchAsync` function. Here is an example of using it to both prefetch data to the currently active GPU device, and then, to the CPU:

```cpp
int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.
```

### Exercise: Prefetch Memory

At this point in the lab, your [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program should not only be launching a CUDA kernel to add 2 vectors into a third solution vector, all which are allocated with `cudaMallocManaged`, but should also be initializing each of the 3 vectors in parallel in a CUDA kernel. If for some reason, your application does not do any of the above, please refer to the following [reference application](../edit/07-init-in-kernel/solutions/01-vector-add-init-in-kernel-solution.cu), and update your own code bases to reflect its current functionality.

Conduct 3 experiments using `cudaMemPrefetchAsync` inside of your [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) application to understand its impact on page-faulting and memory migration.

**Overall Hypothesis**: since prefetching memory will move over entire vectors all at once, and not just when individual pages are requested, the page sizes can be much larger, speeding things up.
When the kernel is actually running, there will be no page faults, since they're already on the device.

- What happens when you prefetch one of the initialized vectors to the device?
    - **Hypothesis**: `initWith()` will be a little faster, since one of the vectors is being fetched async.
    - **Results**: `initWith()` runtime went from 56053441ns to 29334939ns
- What happens when you prefetch two of the initialized vectors to the device?
    - **Hypothesis**: `initWith()` will be even faster.
    - **Results**: `initWith()` runtime went to 14956823ns
- What happens when you prefetch all three of the initialized vectors to the device?
    - **Hypothesis**: `initWith()` go zoom.
    - **Results**: `initWith()` runtime is now 2056023ns, down from 56053441ns **(27x faster!)**

Hypothesize about UM behavior, page faulting specifically, as well as the impact on the reported run time of the initialization kernel, before each experiment, and then verify by running `nsys profile`. Refer to [the solution](../edit/08-prefetch/solutions/01-vector-add-prefetch-solution.cu) if you get stuck.

In [52]:
!nvcc -o prefetch-to-gpu 01-vector-add/01-vector-add.cu -run

Grid size: 40, Block size: 1024
Success! All values calculated correctly.


In [53]:
!nsys profile --stats=true ./prefetch-to-gpu

Grid size: 40, Block size: 1024
Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-785f-270c-8ec9-7dd1.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-785f-270c-8ec9-7dd1.qdrep"
Exporting 1908 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-785f-270c-8ec9-7dd1.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name         
 -------  ---------------  ---------  ----------  -------  ---------  ---------------------
    90.6        240912050          3  80304016.7    17531  240850147  cudaMallocManaged    
     6.9         18399266          3   6133088.7  1411400   15405889  cudaFree             
     1.9          5179613          2   2589806.5  1891105    3288508  cudaDeviceSynchronize
     0.6      

### Exercise: Prefetch Memory Back to the CPU

Add additional prefetching back to the CPU for the function that verifies the correctness of the `addVectorInto` kernel. Again, hypothesize about the impact on UM before profiling in `nsys` to confirm. Refer to [the solution](../edit/08-prefetch/solutions/02-vector-add-prefetch-solution-cpu-also.cu) if you get stuck.

- **Hypothesis**: since we're working with a single vector, page faults will grab large amounts of memory at once anyways, so prefetching shouldn't have much of a benefit (vs when alternating between three vectors).
- **Results**: number of memcpy has gone down, average copy size has gone up, but copy time is virtually unchanged.

In [54]:
!nvcc -o prefetch-to-cpu 01-vector-add/01-vector-add.cu -run

Grid size: 40, Block size: 1024
Success! All values calculated correctly.


In [55]:
!nsys profile --stats=true ./prefetch-to-cpu

Grid size: 40, Block size: 1024
Success! All values calculated correctly.
Processing events...
Saving temporary "/tmp/nsys-report-efe5-0a12-1688-7451.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-efe5-0a12-1688-7451.qdrep"
Exporting 1186 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-efe5-0a12-1688-7451.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name         
 -------  ---------------  ---------  ----------  -------  ---------  ---------------------
    81.8        255796759          3  85265586.3    18869  255732279  cudaMallocManaged    
    13.2         41311200          4  10327800.0    86161   39897607  cudaMemPrefetchAsync 
     3.4         10498837          3   3499612.3   803515    8622524  cudaFree             
     1.6      

After this series of refactors to use asynchronous prefetching, you should see that there are fewer, but larger, memory transfers, and, that the kernel execution time is significantly decreased.

---
## Summary

At this point in the lab, you are able to:

- Use the Nsight Systems command line tool (**nsys**) to profile accelerated application performance.
- Leverage an understanding of **Streaming Multiprocessors** to optimize execution configurations.
- Understand the behavior of **Unified Memory** with regard to page faulting and data migrations.
- Use **asynchronous memory prefetching** to reduce page faults and data migrations for increased performance.
- Employ an iterative development cycle to rapidly accelerate and deploy applications.

In order to consolidate your learning, and reinforce your ability to iteratively accelerate, optimize, and deploy applications, please proceed to this lab's final exercise. After completing it, for those of you with time and interest, please proceed to the *Advanced Content* section.

---
## Final Exercise: Iteratively Optimize an Accelerated SAXPY Application

A basic accelerated SAXPY (Single Precision a\*x+b) application has been provided for you [here](../edit/09-saxpy/01-saxpy.cu). It currently contains a couple of bugs that you will need to find and fix before you can successfully compile, run, and then profile it with `nsys profile`.

After fixing the bugs and profiling the application, record the runtime of the `saxpy` kernel and then work *iteratively* to optimize the application, using `nsys profile` after each iteration to notice the effects of the code changes on kernel performance and UM behavior.

Utilize the techniques from this lab. To support your learning, utilize [effortful retrieval](http://sites.gsu.edu/scholarlyteaching/effortful-retrieval/) whenever possible, rather than rushing to look up the specifics of techniques from earlier in the lesson.

Your end goal is to profile an accurate `saxpy` kernel, without modifying `N`, to run in under *200us*. Check out [the solution](../edit/09-saxpy/solutions/02-saxpy-solution.cu) if you get stuck, and feel free to compile and profile it if you wish.

### Only Bug Fixes

```
CUDA Kernel Statistics:

 Time(%)  Total Time (ns)  Instances   Average    Minimum   Maximum            Name          
 -------  ---------------  ---------  ----------  --------  --------  -----------------------
   100.0         19127823          1  19127823.0  19127823  19127823  saxpy(int*, int*, int*)


CUDA Memory Operation Statistics (by time):

 Time(%)  Total Time (ns)  Operations  Average  Minimum  Maximum              Operation            
 -------  ---------------  ----------  -------  -------  -------  ---------------------------------
    99.7          9104061         580  15696.7     2111   166812  [CUDA Unified Memory memcpy HtoD]
     0.3            24030           4   6007.5     1695    10560  [CUDA Unified Memory memcpy DtoH]


CUDA Memory Operation Statistics (by size in KiB):

   Total    Operations  Average  Minimum  Maximum               Operation            
 ---------  ----------  -------  -------  --------  ---------------------------------
 49152.000         580   84.745    4.000  1000.000  [CUDA Unified Memory memcpy HtoD]
   128.000           4   32.000    4.000    60.000  [CUDA Unified Memory memcpy DtoH]
```

With only bugs fixed: $19232333\texttt{ns}$ = $19232\mu s$

I noticed the largest time consumer in terms of memory operations was HtoD, which also occupied half of the total runtime (9170163ns of the 19232333ns runtime of the `saxpy` kernel).
We can speed this up using prefetching.

### Memory Prefetch to Device

```
CUDA Kernel Statistics:

 Time(%)  Total Time (ns)  Instances  Average   Minimum  Maximum           Name          
 -------  ---------------  ---------  --------  -------  -------  -----------------------
   100.0           194396          1  194396.0   194396   194396  saxpy(int*, int*, int*)


CUDA Memory Operation Statistics (by time):

 Time(%)  Total Time (ns)  Operations  Average   Minimum  Maximum              Operation            
 -------  ---------------  ----------  --------  -------  -------  ---------------------------------
    99.7          8199484          24  341645.2   339897   343577  [CUDA Unified Memory memcpy HtoD]
     0.3            24030           4    6007.5     1695    10560  [CUDA Unified Memory memcpy DtoH]


CUDA Memory Operation Statistics (by size in KiB):

   Total    Operations  Average   Minimum   Maximum               Operation            
 ---------  ----------  --------  --------  --------  ---------------------------------
 49152.000          24  2048.000  2048.000  2048.000  [CUDA Unified Memory memcpy HtoD]
   128.000           4    32.000     4.000    60.000  [CUDA Unified Memory memcpy DtoH]
```

That little optimization alone was enough to get our kernel time down to $194396\texttt{ns}$ = $194\mu s$.
Success!
The number of memcpy operations is way down, with their average size way up.
Grabbing massive pages is the way to go.

### How Much Faster is the Solution Code?

The only other optimization I could think of was optimizing the block size to try and match the number of stream multiprocessors.
I was too lazy to implement it though, so I ran the solution and put the results below for comparison.

```
CUDA Kernel Statistics:

 Time(%)  Total Time (ns)  Instances  Average   Minimum  Maximum           Name          
 -------  ---------------  ---------  --------  -------  -------  -----------------------
   100.0           197596          1  197596.0   197596   197596  saxpy(int*, int*, int*)


CUDA Memory Operation Statistics (by time):

 Time(%)  Total Time (ns)  Operations  Average   Minimum  Maximum              Operation            
 -------  ---------------  ----------  --------  -------  -------  ---------------------------------
    99.7          8180286          24  340845.3   339545   343641  [CUDA Unified Memory memcpy HtoD]
     0.3            24733           4    6183.3     1695    10592  [CUDA Unified Memory memcpy DtoH]


CUDA Memory Operation Statistics (by size in KiB):

   Total    Operations  Average   Minimum   Maximum               Operation            
 ---------  ----------  --------  --------  --------  ---------------------------------
 49152.000          24  2048.000  2048.000  2048.000  [CUDA Unified Memory memcpy HtoD]
   128.000           4    32.000     4.000    60.000  [CUDA Unified Memory memcpy DtoH]
```

Not actually faster at all; most of our runtime was dominated by page faults/fetches anyways, and this micro-optimization clearly has a negligible benefit.

In [86]:
!nvcc -o saxpy 09-saxpy/01-saxpy.cu -run

09-saxpy/01-saxpy.cu(39): warning: variable "multiProcessorCount" was declared but never referenced

Grid size: 16385, Block size: 256
c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 


In [87]:
!nsys profile --stats=true ./saxpy

Grid size: 16385, Block size: 256
c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 
Processing events...
Saving temporary "/tmp/nsys-report-69a8-bdab-9936-0612.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-69a8-bdab-9936-0612.qdrep"
Exporting 1118 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-69a8-bdab-9936-0612.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name         
 -------  ---------------  ---------  ----------  -------  ---------  ---------------------
    95.0        239941835          3  79980611.7    26260  239868491  cudaMallocManaged    
     2.2          5601260          1   5601260.0  5601260    5601260  cudaDeviceSynchronize
     1.8      

In [89]:
!nvcc -o saxpy-solution 09-saxpy/solutions/02-saxpy-solution.cu -run
!nsys profile --stats=true ./saxpy-solution

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 
c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 
Processing events...
Saving temporary "/tmp/nsys-report-ef43-2b9e-0ba0-f803.qdstrm" file to disk...

Creating final output files...
Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-ef43-2b9e-0ba0-f803.qdrep"
Exporting 1104 events: [==================================================100%]

Exported successfully to
/tmp/nsys-report-ef43-2b9e-0ba0-f803.sqlite


CUDA API Statistics:

 Time(%)  Total Time (ns)  Num Calls   Average    Minimum   Maximum           Name         
 -------  ---------------  ---------  ----------  -------  ---------  ---------------------
    94.7        225941762          3  75313920.7    23769  225881841  cudaMallocManaged    
     2.4